In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/id_counts_100_75_higher_Bruce_20190614'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)


In [3]:
daily_item_level_list=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_level_list=[x for x in daily_item_level_list if "Daily" in x]
len(daily_item_level_list)

22

In [4]:
daily_item_level_list[4]

'/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-02/MediaStormDailySales20190205-111300-808.txt'

In [5]:
date_wanted_start="2019-04-18"
date_wanted_end="2019-05-04"

df_daily_item_level_files=pd.DataFrame({"file_path":daily_item_level_list})
df_daily_item_level_files['week_end_dt']=df_daily_item_level_files['file_path'].apply(lambda x: x.split("by_weeks/MediaStorm_")[1][:10])
df_daily_item_level_files=df_daily_item_level_files[df_daily_item_level_files['week_end_dt']>=date_wanted_start]
df_daily_item_level_files=df_daily_item_level_files[df_daily_item_level_files['week_end_dt']<=date_wanted_end]
df_daily_item_level_files.shape

(3, 2)

In [18]:
df_daily_item_level_files['file_path'].tolist()

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-20/MediaStormDailySales20190423-112124-771.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-27/MediaStormDailySales20190430-112453-448.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-04/MediaStormDailySales20190507-113110-141.txt']

In [6]:
df_all_trans_rewards=pd.DataFrame()
df_all_trans_non_rewards=pd.DataFrame()

for file in df_daily_item_level_files['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[(df['transaction_dt']>=date_wanted_start) & (df['transaction_dt']<=date_wanted_end)]
    df['type']=np.where(df['location_id']=="6990","Online","Instore")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_all_trans_rewards=df_all_trans_rewards.append(df_rewards)
    df_all_trans_non_rewards=df_all_trans_non_rewards.append(df_non_rewards)

In [19]:
df.shape

(12960078, 6)

In [7]:
df_all_trans_rewards_agg=df_all_trans_rewards.groupby(['location_id','transaction_dt','transaction_id','customer_id_hashed','type'])['item_transaction_amt'].sum().to_frame().reset_index()
df_all_trans_non_rewards_agg=df_all_trans_non_rewards.groupby(['location_id','transaction_dt','transaction_id','type'])['item_transaction_amt'].sum().to_frame().reset_index()



In [16]:
df_all_trans_non_rewards.shape

(12223357, 6)

In [15]:
df_all_trans_non_rewards_agg=df_all_trans_non_rewards.groupby(['location_id','transaction_dt','transaction_id'])['item_transaction_amt'].sum().to_frame().reset_index()

df_all_trans_non_rewards_agg[df_all_trans_non_rewards_agg['item_transaction_amt']>100].shape

(126032, 4)

In [8]:
df_all_trans_rewards_agg['gruop_75']=np.where(df_all_trans_rewards_agg['item_transaction_amt']>75,"Over75","LE75")
df_all_trans_rewards_agg['gruop_100']=np.where(df_all_trans_rewards_agg['item_transaction_amt']>100,"Over100","LE100")

df_all_trans_rewards_agg['gruop_75_type']=df_all_trans_rewards_agg['gruop_75']+"|"+df_all_trans_rewards_agg['type']
df_all_trans_rewards_agg['gruop_100_type']=df_all_trans_rewards_agg['gruop_100']+"|"+df_all_trans_rewards_agg['type']

df_all_trans_non_rewards_agg['gruop_75']=np.where(df_all_trans_non_rewards_agg['item_transaction_amt']>75,"Over75","LE75")
df_all_trans_non_rewards_agg['gruop_100']=np.where(df_all_trans_non_rewards_agg['item_transaction_amt']>100,"Over100","LE100")

df_all_trans_non_rewards_agg['gruop_75_type']=df_all_trans_non_rewards_agg['gruop_75']+" | "+df_all_trans_non_rewards_agg['type']
df_all_trans_non_rewards_agg['gruop_100_type']=df_all_trans_non_rewards_agg['gruop_100']+" | "+df_all_trans_non_rewards_agg['type']

In [9]:
output_75_rewards=df_all_trans_rewards_agg.groupby(['gruop_75']).agg({"customer_id_hashed":"nunique","transaction_id":"count"}).reset_index()
output_100_rewards=df_all_trans_rewards_agg.groupby(['gruop_100']).agg({"customer_id_hashed":"nunique","transaction_id":"count"}).reset_index()

output_75_type_rewards=df_all_trans_rewards_agg.groupby(['gruop_75','type']).agg({"customer_id_hashed":"nunique","transaction_id":"count"}).reset_index()
output_100_type_rewards=df_all_trans_rewards_agg.groupby(['gruop_100','type']).agg({"customer_id_hashed":"nunique","transaction_id":"count"}).reset_index()


output_rewards=output_75_rewards.append(output_100_rewards).append(output_75_type_rewards).append(output_100_type_rewards)

In [10]:
output_75_non_rewards=df_all_trans_non_rewards_agg.groupby(['gruop_75']).agg({"transaction_id":"count"}).reset_index()
output_100_non_rewards=df_all_trans_non_rewards_agg.groupby(['gruop_100']).agg({"transaction_id":"count"}).reset_index()

output_75_type_non_rewards=df_all_trans_non_rewards_agg.groupby(['gruop_75','type']).agg({"transaction_id":"count"}).reset_index()
output_100_type_non_rewards=df_all_trans_non_rewards_agg.groupby(['gruop_100','type']).agg({"transaction_id":"count"}).reset_index()


output_non_rewards=output_75_non_rewards.append(output_100_non_rewards).append(output_75_type_non_rewards).append(output_100_type_non_rewards)

In [11]:
output_rewards=output_rewards[['type','gruop_100','gruop_75','transaction_id','customer_id_hashed']]
output_non_rewards=output_non_rewards[['type','gruop_100','gruop_75','transaction_id']]

In [12]:
writer=pd.ExcelWriter("./BL_id_counts_2019-04-18_to_2019-05-04_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_rewards.to_excel(writer,"rewards",index=False)
output_non_rewards.to_excel(writer,"non_rewards",index=False)
writer.save()